In [13]:
import pandas as pd
import gurobipy   as      gp
from   gurobipy   import GRB
from MasterProblemOMP import MasterProblem
from OMPopenPit import SubProblem

In [2]:
#path = "C:/Users/willi/OneDrive/Escritorio/Magister/Tesis-Magister/Database/integratedModel/" #Notebook
path = "/home/williams/Tesis-Magister/Database/integratedModel/"
#path = "C:/Users/Williams Medina/Desktop/Tesis Magister/Tesis-Magister/ThesisCode/MainCode/Databases/integratedModel/" #Desktop
undergroundDatabaseName = 'Modelo_F_OG.xlsx'
#openPitDatabaseName = 'Modelo_F_OG.xlsx'
openPitDatabaseName = 'Modelo_F_OG_4_4_4.xlsx'

In [3]:
undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
openPitDataframe = pd.read_excel(path + openPitDatabaseName, engine="openpyxl") #Notebook

In [11]:
class Main:
    def __init__(self, undergroundMineDataframe, openPitDataframe):
        self.openPitDataframe = openPitDataframe
        self.undergroundMineDataframe = undergroundMineDataframe
        self.numberOfPeriods = 5
        self.epsilon = 1000
        
    def execute(self):
        self.createModels()
        self.setMasterProblemCrownPillarHeights()
        self.getResults()
    
    def createModels(self):
        self.createSubProblem()
        self.createMasterProblem()

    def createSubProblem(self):
        self.SubProblem = SubProblem(self.openPitDataframe, self.numberOfPeriods)

    def createMasterProblem(self):
        self.MasterProblem = MasterProblem(self.undergroundMineDataframe, self.numberOfPeriods)
        self.MasterProblem.setParameters()
    
    def setMasterProblemCrownPillarHeights(self):
        self.MasterProblem.V, self.MasterProblem.rho_v = self.SubProblem.V, self.SubProblem.rho_v
        self.MasterProblem.B_v = self.SubProblem.B_v


    def getResults(self):
        it = 0
        self.MasterProblem.setModel()
        currentTheta = -100000000000
        while True:
            it +=1
            self.estimatedW_v, self.estimatedTheta = self.MasterProblem.optimize()
            #self.estimatedW_v = {key:value.X for key,value in zip(self.estimatedW_v, self.estimatedW_v.values())}
            subProblemObjValue, pi_vb = self.SubProblem.optimize(self.estimatedW_v)
            #Cuál es la condición de termino?, en el seudo algoritmo es dif de 
            """if subProblemObjValue - self.MasterProblem.theta.X <= self.epsilon:
                print("Optimal Crown Pillar height: {}".format(self.estimatedW_v))
                print("Optimal ObjVal: {}".format(subProblemObjValue + self.MasterProblem.undergroundModel.objVal))
                print("Optimal Theta: {}".format(self.MasterProblem.theta.X))
                #print("Current theta: {}".format(currentTheta))
                break"""
            if abs(self.MasterProblem.theta.X - currentTheta) <= self.epsilon:
                print()
                print("-----------------------------Valor óptimo encontrado-----------------------------")
                print("Optimal Crown Pillar height: {}".format(self.estimatedW_v))
                print("Optimal ObjVal: {}".format(subProblemObjValue + self.MasterProblem.undergroundModel.objVal))
                print("Optimal Theta: {}".format(self.MasterProblem.theta.X))
                print("Current theta: {}".format(currentTheta))
                break
            
            print(it, "Objective Value =", self.MasterProblem.undergroundModel.objVal)
            print("w_v: {}".format(self.estimatedW_v))
            print("Master theta: {}".format(self.MasterProblem.theta))
            print("Current theta: {}".format(currentTheta))
            currentTheta = self.estimatedTheta.X
            self.MasterProblem.undergroundModel.addConstr(self.MasterProblem.theta <= subProblemObjValue + gp.quicksum(gp.quicksum((self.MasterProblem.w_v[v]-self.estimatedW_v[v]) * pi_vb[b] for b in self.SubProblem.B_v) for v in self.SubProblem.V))
            #self.MasterProblem.undergroundModel.addConstr(self.MasterProblem.w_v[1265] == 1)
            #self.SubProblem.openPitModel.addConstrs(gp.quicksum(self.SubProblem.x_bt[ti, b] for ti in self.SubProblem.t_C) <= 1 - self.estimatedW_v[v] for v in self.SubProblem.V for b in self.SubProblem.B_v[v])
            #self.MasterProblem.addThetaRestriction(subProblemObjValue,self.estimatedW_v,pi_vb)
            print()
            print()

In [12]:
main = Main(undergroundMineDataframe, openPitDataframe)
main.execute()

Set parameter TimeLimit to value 3600
1 Objective Value = 804143376.9373022
w_v: {745: 1.0, 785: 0.0, 825: 0.0, 865: 0.0, 905: 0.0, 945: 0.0, 985: 0.0, 1025: 0.0, 1065: 0.0, 1105: 0.0, 1145: 0.0, 1185: 0.0, 1225: 0.0, 1265: 0.0, 1305: 0.0}
Master theta: <gurobi.Var theta (value 800000000.0)>
Current theta: -100000000000
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [7]:
main.MasterProblem.theta.X - (-100000000000) <= main.epsilon

False

In [8]:
750622680-735921374.0472324

14701305.95276761

In [9]:
main.MasterProblem.undergroundModel.objVal

739375340.174233

In [10]:
main.SubProblem.openPitModel.objVal

735231963.2369306